# Checking Pastas models

*Developed by D.A. Brakenhoff, Artesia*

This notebooks showcases the `pastas.check` submodule. This module can be used to check your time series models with pre-defined checks, or users can define their own checks.

For more elaborate discussions on why checking different aspects of your model is a good idea, we refer you to some of the other notebooks in the documentation, e.g. [diagnostic_checking.ipynb](https://pastas.readthedocs.io/latest/examples/diagnostic_checking.html) and/or [stowa_calibration.ipynb](https://pastas.readthedocs.io/latest/examples/stowa_calibration.html). In this notebook we focus on the `pastas.check` tools.

In [ ]:
import pandas as pd

import pastas as ps

## Build a simple model

First let's load some head data and build a simple time series model. 

In [ ]:
obs = pd.read_csv("data/head_nb1.csv", index_col=0, parse_dates=True).squeeze("columns")
rain = pd.read_csv("data/rain_nb1.csv", index_col=0, parse_dates=True).squeeze(
    "columns"
)
evap = pd.read_csv("data/evap_nb1.csv", index_col=0, parse_dates=True).squeeze(
    "columns"
)

We use a linear recharge model, so $R = P - f \cdot E$. Build the model and solve it.

In [ ]:
ml = ps.Model(obs, name="groundwater_head")
sm = ps.RechargeModel(prec=rain, evap=evap, rfunc=ps.Gamma(), name="recharge")
ml.add_stressmodel(sm)
ml.solve()

In [ ]:
ax = ml.plot(figsize=(10, 3))

## Checking the model

The pastas `check` module contains check functions and a convenience function if you
want to do multiple checks on a single model.

Let's inspect the module:

In [ ]:
ps.check?

### What is a check function?

Check functions are the methods that determine whether a pastas Model meets certain
criteria (passes the check) or fails to meet those criteria (fails the check). The
criteria can be anything, from goodness-of-fit statistics to the magnitude of the
variation of some contribution to the head. It is up to the modeller to decide which
criteria their model should be subjected to.

Check functions follow a specific template:

 - A `pastas.Model` as the first argument
 - Any number of keyword arguments after that
 - Return a DataFrame containing information about the performed check. The following columns are defined:
    - **statistic:** the test statistic
    - **operator:** the operator, e.g. >, ==, within, etc.
    - **threshold:** the user-specified threshold or comparison values
    - **dimensions:** units or dimensions of statistic (useful for interpretation)
    - **pass:** whether the check is passed or not
    - **comment:** any additional comments relevant to the check

### Pre-defined checks

The `pastas.check` module contains several pre-defined checks that are frequently
applied to time series models. These can be listed with `ps.check.checks` which is a
dictionary containing the names and functions.

In [ ]:
list(ps.check.checks.keys())

#### Check: $R^2 \geq$ threshold

Let's try applying the `rsq_geq_threshold` check to our model, which tests whether the
$R^2 \geq s$, where $s$ is some user-defined threshold. We can already see the $R^2$
fit in the plot of the model simulation above, so let's say we want our fit to be
greater or equal to 0.9. The model should obviously pass this check.

In [ ]:
ps.check.rsq_geq_threshold(ml, threshold=0.9)

This example isn't very interesting, but you can already
see that when we want to apply multiple checks to all of our models, having each of
these checks contained within a function can be useful.

### Writing your own checks

Now as an extra example let's write our own check function. This time we want to check
whether our model is sufficiently good at simulating the low groundwater levels in
summer.

We'll do this by checking the goodness-of-fit for the summer periods. We can write a
function that accepts a list of months in which to consider the residuals in order to
calculate the fit statistics.

In [ ]:
def rsq_geq_threshold_in_months(
    ml: ps.Model, threshold: float, months: list[int]
) -> pd.DataFrame:
    """Check if the R² of the model is >= to a threshold in specific months.

    Parameters
    ----------
    ml : pastas.Model
        The Pastas model to check.
    threshold : float
        The R² threshold value.
    months : list of ints
        The month numbers to consider in each year.

    Returns
    -------
    pd.DataFrame
        A DataFrame showing the check results
    """
    res = ml.residuals()  # get the model residuals
    mask = res.index.month.isin(months)  # define a mask for the selected months
    rsq = ps.stats.rsq(obs=ml.observations().loc[mask], res=res.loc[mask])  # compute R²
    # store results and context
    df = ps.check.get_empty_check_dataframe()
    df.loc["rsq_geq_threshold_in_months"] = (
        rsq,
        ">=",
        threshold,
        "-",
        rsq >= threshold,
        f"in months: {months}",
    )
    return df

In [ ]:
rsq_geq_threshold_in_months(ml, threshold=0.8, months=[6, 7, 8])

## Applying multiple checks to a model

Often we want to check multiple criteria when deciding whether our model is fit for
purpose. 

### Pre-defined checks

Some of the pre-defined checks already check multiple criteria, for example whether
the parameters do not lie on the parameter bounds. These methods return a DataFrame
with multiple rows, with the results of each check on a separate row.

#### Check: Parameter bounds

This check is automatically performed when solving a time series model, and any warnings are logged and reported at the bottom of the fit report. But it can be useful to include it in your list of checks. When an optimal parameter lies on a boundary, the check fails.

In [ ]:
ps.check.parameter_bounds(ml)

#### Check: Uncertainty parameters

This check tests whether the absolute value of the optimal parameters is larger than some factor times the estimated standard deviation of the parameter. Basically, it checks whether the parameter can be estimated with sufficient certainty. This check requires that the estimate of $\sigma$ is reliable, which is the case when the noise meets certain requirements. This check always includes this comment, to remind users to check the noise prior to trusting the estimated parameter uncertainties.

In [ ]:
ps.check.uncertainty_parameters(ml, n_std=1.96)

#### Check: Length response relative to calibratiod period or warmup

The memory of the response function indicates the time it takes until the effect of some change has taken place. For example, the time it takes for the groundwater level to rise after 1 mm of rain today. This time is commonly expressed as a percentage e.g. $t_{95}$, the time it takes until 95% of the rise has taken place. (Since the response is asymptotic, the 100% lies at time infinitiy.)

This memory should never be longer than the calibration period, and preferably be significantly shorter. By default the $t_{95}$ is used and it is compared to half the length of the calibration period.

In [ ]:
ps.check.response_memory(ml, cutoff=0.95, factor_length_oseries=0.5)

The memory can also compared to the warmup period. When the memory exceeds the warmup period, the model is not yet done warming up by the time the simulation starts, which can cause issues when calibrating or simulating the model for different time periods.

In [ ]:
ps.check.response_memory_vs_warmup(ml, cutoff=0.95)

### The `checklist` function for performing multiple checks

The function `ps.check.checklist` is a convenience function for applying multiple
checks to a model. This function accepts a list of checks. This list can consist of the
following items:
   
   - the name of a built-in check, e.g. "rsq_geq_threshold"
   - any function that requires only a model as its input
   - a dictionary containing any function and any additional arguments to be passed to the check function.

Note that in the first two cases it is not possible to alter any additional arguments
to the functions. If relevant it will use default values. Additionally, each check
function should return a DataFrame with the expected columns, so the results can be
combined.

The example below show-cases the application of 3 checks in each of the ways described
above. Optionally, a report is shown (basically the resulting DataFrame) in which the
checks (pass/fail) are colored for quick visual inspection.

In [ ]:
# add checks to a list
checklist = [
    # name of a built-in check
    "rsq_geq_threshold",
    # any check function that only requires the model as input
    ps.check.parameter_bounds,
    # check function, with additional arguments
    {
        "func": rsq_geq_threshold_in_months,
        "threshold": 0.8,
        "months": [6, 7, 8],
    },
]

# perform all checks on a model, and optionally display a report
checks = ps.check.checklist(ml, checklist, report=True)

## Checks in literature

Currently, the `ps.check` module contains two pre-defined checklists taken from two articles on time series analysis: 

- the checks used in Brakenhoff et al. (2022).
- the checks used in Zaadnoordijk et al. (2019).

These articles used certain checks to determine whether a time series model was reliable or not.

These checklists can be passed to the `ps.check.checklist()` function. Note that some of these checks require the noise to meet the requirement of white noise. Our model currently does not have a noise model, so let's see what that does to the checks.

In [ ]:
checks_no_noise = ps.check.checklist(ml, ps.check.checks_brakenhoff_2022)

Now let's add a noisemodel, solve the model again and run the checks again.

In [ ]:
ml.add_noisemodel(ps.ArNoiseModel())
ml.solve(initial=False)

Note that the autocorrelation check now passes, which means we are more confident that the estimated parameter uncertainties are reliable.

*__Note:__ No significant autocorrelation is just one of the criteria the noise has to meet in order to meet the requirements of white noise, but we will not discuss that here. For more information refer to the notebooks [diagnostic_checking.ipynb](https://pastas.readthedocs.io/latest/examples/diagnostic_checking.html) and/or [stowa_calibration.ipynb](https://pastas.readthedocs.io/latest/examples/stowa_calibration.html).*

In [ ]:
check_w_noise = ps.check.checklist(ml, ps.check.checks_brakenhoff_2022)

## References

- Brakenhoff DA, Vonk MA, Collenteur RA, Van Baar M and Bakker M (2022) Application of Time Series Analysis to Estimate Drawdown From Multiple Well Fields. Front. Earth Sci. 10:907609. [doi: 10.3389/feart.2022.907609](https://www.frontiersin.org/journals/earth-science/articles/10.3389/feart.2022.907609/full)
- Zaadnoordijk, W.J., Bus, S.A.R., Lourens, A. and Berendrecht, W.L. (2019), Automated Time Series Modeling for Piezometers in the National Database of the Netherlands. Groundwater, 57: 834-843. https://doi.org/10.1111/gwat.12819